In [6]:
!pip install plotly
!pip install pandas
!pip install geopy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.0 MB/s eta 0:00:00


In [10]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from geopy.geocoders import Nominatim

# Load the dataset into a DataFrame
data = pd.read_csv('covidnewpractice.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Create a geocoder instance
geolocator = Nominatim(user_agent='heatmap_app', timeout=10)

# Create a figure
fig = go.Figure()

# Iterate over unique recorded dates
for date in data['date'].dt.date.unique():
    # Filter data for the specific date
    filtered_data = data[data['date'].dt.date == date]

    # Convert country names to latitude and longitude coordinates
    coordinates = filtered_data['location'].apply(lambda x: geolocator.geocode(x).point if geolocator.geocode(x) else None)
    filtered_data['latitude'] = [coord[0] if coord else None for coord in coordinates]
    filtered_data['longitude'] = [coord[1] if coord else None for coord in coordinates]

     # Drop rows with missing coordinates
    filtered_data = filtered_data.dropna(subset=['latitude', 'longitude'])
    
    # Add a heatmap trace for the date
    
    # Add a heatmap trace for the date
    fig.add_trace(go.Densitymapbox(
        lat=filtered_data['latitude'],
        lon=filtered_data['longitude'],
        z=filtered_data['new_cases_smoothed'],  # Your value column for color intensity
        radius=10,
        coloraxis="coloraxis"
    ))

# Set up the layout
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center_lon=0,
    mapbox_center_lat=0,
    mapbox_zoom=0,
    coloraxis=dict(colorscale='Viridis', cmin=0, cmax=data['value'].max())
)

# Create a slider to control the date
slider = dict(
    active=0,
    steps=[go.layout.slider.Step(method='restyle', args=[{'visible': [True if i == j else False for i in range(len(fig.data))]}],
                                 label=str(date.date()), value=str(date.date()))
           for j, date in enumerate(data['date'].dt.date.unique())]
)

# Update the layout with the slider
fig.update_layout(sliders=[slider])

# Export the figure as an HTML file
pio.write_html(fig, 'recent_covid_heatmap.html')


/tmp/ipykernel_3928/1685378651.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3928/1685378651.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



GeocoderServiceError: Non-successful status code 502

In [11]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from geopy.geocoders import Nominatim

# Load the dataset into a DataFrame
data = pd.read_csv('covidnewpractice.csv')

# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Create a geocoder instance
geolocator = Nominatim(user_agent='heatmap_app', timeout=10)

# Create a figure
fig = go.Figure()

# Iterate over unique recorded dates
for date in data['date'].dt.date.unique():
    # Filter data for the specific date
    filtered_data = data[data['date'].dt.date == date].copy()

    # Convert country names to latitude and longitude coordinates using .loc
    filtered_data.loc[:, 'latitude'] = filtered_data['location'].apply(
        lambda x: geolocator.geocode(x).point[0] if geolocator.geocode(x) else None
    )
    filtered_data.loc[:, 'longitude'] = filtered_data['location'].apply(
        lambda x: geolocator.geocode(x).point[1] if geolocator.geocode(x) else None
    )

    # Drop rows with missing coordinates
    filtered_data = filtered_data.dropna(subset=['latitude', 'longitude'])
    
    # Add a heatmap trace for the date
    fig.add_trace(go.Densitymapbox(
        lat=filtered_data['latitude'],
        lon=filtered_data['longitude'],
        z=filtered_data['new_cases_smoothed'],  # Your value column for color intensity
        radius=10,
        coloraxis="coloraxis"
    ))

# Set up the layout
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_center_lon=0,
    mapbox_center_lat=0,
    mapbox_zoom=0,
    coloraxis=dict(colorscale='Viridis', cmin=0, cmax=data['new_cases_smoothed'].max())
)

# Create a slider to control the date
slider = dict(
    active=0,
    steps=[go.layout.slider.Step(method='restyle', args=[{'visible': [True if i == j else False for i in range(len(fig.data))]}],
                                 label=str(date.date()), value=str(date.date()))
           for j, date in enumerate(data['date'].dt.date.unique())]
)

# Update the layout with the slider
fig.update_layout(sliders=[slider])

# Export the figure as an HTML file
fig.write_html('recent_covid_heatmap.html')


GeocoderServiceError: Non-successful status code 502